In [1]:
import asyncio

In [2]:
"""
先用最简单的方式实现：
    thread的启停（使用state?保存函数名称，在外部。
    timer的加入
    button sync
    list button
"""
None

In [3]:
class State:
    funclist=[]    #for TaskHandler


In [4]:
class TaskHandler:
    def __init__(self,func,*args,**kwargs):
        self.func=func
        self.task=None
        self.args=args
        self.kwargs=kwargs
    def start(self,l=None):
        if not self.is_running:
            self.task=asyncio.create_task(self.func(*self.args,**self.kwargs))
    
    error=None
    @property
    def is_running(self):
        if self.task is None:return False
        if not self.task.done():return True
        self.end()
        del self.task
        self.task=None
        
        
    
    def cancel(self):
        if not self.is_running:return
        self.task.cancel()
        
            
    def stop(self,l=None):
        self.cancel()
        
    def end(self):
        try:self.task.result()
        except asyncio.CancelledError:pass
        except asyncio.InvalidStateError:print("result not good")
        except Exception as e:self.error=e

    name=""
    def __str__(self):
        return self.name

In [5]:
from functools import wraps
def add_to_state(state):
    def wrapper(func):
        @wraps(func)
        def wrapped(*args, **kwargs):
            state.funclist.append(TaskHandler(func,*args, **kwargs))
            return
        return wrapped
    return wrapper

In [6]:
class StateT(State):
    def __init__(self):
        print("""info
        
        Always be aware that this version of dashboard is instanced when defination.
        Which means that there cannot be two dashboard in the same ipynb kernel.
        
        """)
    def find_func_by_name(self,name):
        for i in self.funclist:
            if i.name==name:return i

    def start_func(self):
        ...
    
    def display(self):
        for i in self.funclist:
            i.display()


s=StateT()

info
        
        Always be aware that this version of dashboard is instanced when defination.
        Which means that there cannot be two dashboard in the same ipynb kernel.
        
        


In [7]:
import ipywidgets
class TaskHandlerButton(TaskHandler):
    def __init__(self,name,state,func,widgets,*args,**kwargs):
        self.name=name
        self.state=state        
        
        self.func=func
        self.task=None
        self.args=[self,*args]
        self.kwargs=kwargs

        self.widgets=widgets(self)
        self.create_buttons(self.widgets)
        
    def change_name(self,name):
        self.name=name
        self.name_button.description=self.name
    
    @property
    def default_layout(self):
        return ipywidgets.Layout(width='auto', height='30px')
    
    def button(self,*args,**kwargs):
        if "layout" not in kwargs:
            kwargs["layout"]=self.default_layout
        return ipywidgets.Button(*args,**kwargs)
    def wfloat(self,*args,**kwargs):
        if "layout" not in kwargs:
            kwargs["layout"]=self.default_layout
        return ipywidgets.BoundedFloatText(*args,**kwargs)
    def wtext(self,*args,**kwargs):
        if "layout" not in kwargs:
            kwargs["layout"]=self.default_layout
        return ipywidgets.Text(*args,**kwargs)
    
    def create_buttons(self,widgets):
        self.name_button = self.button(description=self.name,disabled=True)
        self.start_button = self.button(description="start")
        self.start_button.on_click(self.start)
        self.stop_button = self.button(description="stop")
        self.stop_button.on_click(self.stop)
        
        self.start_button.default=True
        self.stop_button.default=True
        
        # self.box=ipywidgets.Box([self.start_button,self.stop_button])
        default_buttons=self.name_button,self.start_button,self.stop_button
        # (setattr(i,"default",True) for i in default_buttons)
        self.box=ipywidgets.Box([*default_buttons,*widgets])
        
    def display(self):
        display(self.box)

def add_to_state_button(state,widgets):
    def wrapper(func):
        @wraps(func)
        def wrapped(name,*args, **kwargs):
            state.funclist.append(TaskHandlerButton(name,state,func,widgets,*args, **kwargs))
            return
        return wrapped
    return wrapper

In [8]:
async def sync_value(from_value,to_value,
                     from_value_name="value",to_value_name="value",
                     sync_time=0.01):
    while 1:
        await asyncio.sleep(sync_time)
        if hasattr(to_value,to_value_name) and hasattr(from_value,from_value_name):break
    while 1:
        await asyncio.sleep(sync_time)
        x=getattr(from_value,from_value_name)
        if x==getattr(to_value,to_value_name):continue
        setattr(to_value,to_value_name,x)



In [9]:
def text_widgets(task):
    return (task.wfloat(),)
@add_to_state_button(s,text_widgets)
async def loop_timer(task,sleep_interval=0.1,keep_len=10):
    #the variable used is called "loop_time"
    
    #TODO: better timing function to reduce execution time
    from collections import deque
    import time
    timed_list=deque((),keep_len)

    #state.loop_time=-1
    #old version no load loop time:1.0171284675598145
    #used "asyncio.create_task(sync_value(s,t,"loop_time"))" at outside
    
    #new version no load loop time:1.0043690204620361
    text_time=task.widgets[0]
    
    
    for _ in range(keep_len):
        await asyncio.sleep(sleep_interval)
        timed_list.append(time.time())
        
    while 1:
        await asyncio.sleep(sleep_interval)
        t=time.time()
        text_time.value=t-timed_list.popleft()
        timed_list.append(t)
        
loop_timer("loop_timer")

In [10]:
s.funclist

In [11]:
s.funclist[0].is_running

False

In [12]:
s.funclist[0].task